In [1]:
import torch
import json
from tqdm import tqdm
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
from dataset import Dataset
from utils import load_model, get_all_run_ids, get_device

In [2]:
models_path = "../trained/transformer-autoencoder-timecomp-jan"
run_ids = get_all_run_ids(path=models_path)
device = get_device()
models = {}
for _id in run_ids:
    models[_id] = load_model(_id, path=models_path).to(device)



In [5]:
dataset = Dataset().load_dataset_from_pickle(pickle_path="../../dataset/data/0117/0117_processed_1024.pkl")
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [17]:
max, min = torch.empty(0).to("cpu"), torch.empty(0).to("cpu")
models_range = {}

with torch.no_grad():
    for _id in run_ids:
        model = models[_id]
        
        _max, _min = torch.empty(0).to("cpu"), torch.empty(0).to("cpu")

        for batch_idx, (train_inputs, _) in enumerate(tqdm(dataloader)):
            out = model.forward_encoder(train_inputs.to(device)).detach().to("cpu")
            
            _max = torch.cat((_max, out.max(dim=1).values.max(dim=0).values.unsqueeze(0)), dim=0)
            _min = torch.cat((_min, out.min(dim=1).values.min(dim=0).values.unsqueeze(0)), dim=0)
            
            models_range[_id] = (_max, _min)
            
        _max = _max.max(dim=0).values     
        _min = _min.min(dim=0).values
               
        torch.cuda.empty_cache()
        
        models_range[_id] = {"min":_min.tolist(), "max": _max.tolist()}
        
        print("model", _id, "max", _max.tolist(), "min", _min.tolist())
            
        max = torch.cat((max, _max.unsqueeze(0)), dim=0)
        min = torch.cat((min, _min.unsqueeze(0)), dim=0)
        

100%|██████████| 1614/1614 [00:02<00:00, 611.78it/s]


model 0daq0g9h max [2.7029311656951904, 2.1502432823181152, 2.322965383529663, 2.1830039024353027] min [-2.581634998321533, -1.9115527868270874, -3.1803038120269775, -2.339738607406616]


100%|██████████| 1614/1614 [00:03<00:00, 429.89it/s]


model sys6hs5g max [2.301015615463257, 2.3510663509368896, 1.5381203889846802, 1.894698143005371] min [-2.2298760414123535, -1.7534922361373901, -1.6228348016738892, -1.8588333129882812]


100%|██████████| 1614/1614 [00:02<00:00, 736.60it/s]

model 8j4cqfd8 max [0.7843466997146606, 0.7664592266082764, 0.8660548329353333, 0.694176971912384] min [-0.8101696372032166, -0.8937942385673523, -0.8518332839012146, -0.7511453628540039]


In [18]:
models_range

{'0daq0g9h': {'min': [-2.581634998321533,
   -1.9115527868270874,
   -3.1803038120269775,
   -2.339738607406616],
  'max': [2.7029311656951904,
   2.1502432823181152,
   2.322965383529663,
   2.1830039024353027]},
 'sys6hs5g': {'min': [-2.2298760414123535,
   -1.7534922361373901,
   -1.6228348016738892,
   -1.8588333129882812],
  'max': [2.301015615463257,
   2.3510663509368896,
   1.5381203889846802,
   1.894698143005371]},
 '8j4cqfd8': {'min': [-0.8101696372032166,
   -0.8937942385673523,
   -0.8518332839012146,
   -0.7511453628540039],
  'max': [0.7843466997146606,
   0.7664592266082764,
   0.8660548329353333,
   0.694176971912384]}}

In [20]:
with open("../../models/trained/transformer-autoencoder-timecomp-jan/models_range.json", "wb") as f:
    json.dump(models_range, f)

TypeError: a bytes-like object is required, not 'str'